# Imports
===================================================================================================================================================================================

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

# Loading and Combining data
=============================================================================================================================================

The data from each of the police forces come in the following format,

- raw-data
    - 2023-06
        - 2023-06-essex-street.csv
        - 2023-06-essex-stop-and-search.csv
            ...


The aim of the code below is to combine csv files so that we have one csv file for each datatype. This means the data from all 5 people forces will be collected together also.

- combined-data
    - combined_allregions_street.csv
    - combined_allregions_stop-and-search.csv


In [2]:
## Functions for loading data
def combine_all_regions_by_datatype(datatypes,
                                    root_folder,
                                    output_folder,
                                    file_pattern = "*.csv"
                                    ):
    """
    Combine CSVs across **all regions** for each datatype in `datatypes`,
    under `root_folder`, writing one output file per datatype into `output_folder`.

    Parameters:
      datatypes     : list of datatypes e.g. ["street","outcomes","stop-and-search"]
      root_folder   : folder path containing sub-folders by month/year
      file_pattern  : pattern for CSV files (default "*.csv")
      output_folder : folder to write combined results (if None, uses root_folder)

    Returns:
      List of paths to the combined CSV files created (one per datatype).
    """
    if output_folder is None:
        output_folder = root_folder

    os.makedirs(output_folder, exist_ok=True)

    datatypes_lower = [dt.strip().lower() for dt in datatypes]

    # Find all files once
    search_path = os.path.join(root_folder, "**", file_pattern)
    all_files = glob.glob(search_path, recursive=True)

    output_paths = []

    for datatype, datatype_lower in zip(datatypes, datatypes_lower):
        matching_files = []
        for fpath in all_files:
            fname = os.path.basename(fpath)
            name_no_ext, _ = os.path.splitext(fname)
            parts = name_no_ext.split('-')
            if len(parts) >= 4:
                remaining = parts[2:]  # region + datatype parts
                dt_parts = datatype_lower.split('-')
                if remaining[-len(dt_parts):] == dt_parts:
                    # Accept this file for this datatype (regardless of region)
                    matching_files.append(fpath)

        if not matching_files:
            print(f"[WARNING] No files found for datatype '{datatype}'. Skipping.")
            continue

        # Read & combine
        df_list= []
        for fp in matching_files:
            try:
                df = pd.read_csv(fp)
                parts = os.path.splitext(os.path.basename(fp))[0].split('-')
                # The region is everything between parts[2] to parts[-len(dt_parts)-1]
                region_part = "-".join(parts[2:-len(dt_parts)])
                df['Region']   = region_part
                df_list.append(df)
            except Exception as exc:
                print(f"Warning: failed to read '{fp}': {exc}")

        if not df_list:
            print(f"[ERROR] No readable CSVs for datatype '{datatype}'. Skipping.")
            continue

        combined_df = pd.concat(df_list, ignore_index=True)
        out_fname = f"combined_allregions_{datatype}.csv"
        out_path  = os.path.join(output_folder, out_fname)
        combined_df.to_csv(out_path, index=False)
        output_paths.append(out_path)
        print(f"Combined file for datatype '{datatype}' written to: {out_path}")

    return output_paths


Calling the function just made, and producing 2 new csv files with the combined data

In [3]:
root_folder = "raw-data"
datatypes   = ["street", "stop-and-search"]
output_folder = "combined-data"

combined_paths = combine_all_regions_by_datatype(
    datatypes     = datatypes,
    root_folder   = root_folder,
    output_folder = output_folder
)


Combined file for datatype 'street' written to: combined-data\combined_allregions_street.csv
Combined file for datatype 'stop-and-search' written to: combined-data\combined_allregions_stop-and-search.csv


Turning the csv files into dataframes

In [4]:
# Read New csv files and save them to dataframes
df_street = pd.read_csv(combined_paths[0])
df_stop_and_search = pd.read_csv(combined_paths[1])

We will be looking mainly at geospatial data, so we need to load our region boundaries to use them as a filter later

In [5]:
# 1. Load Region boundaries from KML files
boundaries = []
kml_folder = 'force-kmls'
kml_files = [os.path.join(kml_folder, f) for f in os.listdir(kml_folder) if f.endswith('.kml')]

for file in kml_files:
    try:
        gdf = gpd.read_file(file, driver='KML')
        boundaries.append(gdf)
    except Exception as e:
        print(f"Could not read {file}: {e}")

final_gdf = gpd.GeoDataFrame(pd.concat(boundaries, ignore_index=True), crs="EPSG:4326")


# Preprocessing
==============================================================================================================================================================
## Dataframe - Street
---------------
- Copy of raw data created
- Having a look at the shape of df_street, and all the column headers


In [6]:
df_street_raw = df_street.copy()

## Shape and columns of Dataframe - Street
print(f"Dataframe-street:\n shape:{df_street.shape} \n column header old:{df_street.columns.to_list()}" )

## Preview of Dataframe - Street
df_street.head()


Dataframe-street:
 shape:(1468218, 13) 
 column header old:['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type', 'Last outcome category', 'Context', 'Region']


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Region
0,NaN,2023-10,Essex Police,Essex Police,0.439004,51.641536,On or near Derby Close,E01021237,Basildon 001A,Anti-social behaviour,NaN,NaN,essex
1,663fc84dec3c68ead7dbdc274183432ad72008d9c1eb89...,2023-10,Essex Police,Essex Police,0.439614,51.639167,On or near Smythe Road,E01021237,Basildon 001A,Other theft,Investigation complete; no suspect identified,NaN,essex
2,446d4dc29575f600213a210fe477eb2dee5ccad548b3f2...,2023-10,Essex Police,Essex Police,0.432091,51.640106,On or near Oakwood Drive,E01021237,Basildon 001A,Violence and sexual offences,Further action is not in the public interest,NaN,essex
3,04246cd15ed95c5a8db98467158ff84c16a3944042300e...,2023-10,Essex Police,Essex Police,0.435584,51.640825,On or near Doublet Mews,E01021238,Basildon 001B,Vehicle crime,Investigation complete; no suspect identified,NaN,essex
4,1d720ddc539b23c226a96b67a52a9cba67b89007cd6230...,2023-10,Essex Police,Essex Police,0.423868,51.644206,On or near Sussex Way,E01021250,Basildon 001D,Violence and sexual offences,Unable to prosecute suspect,NaN,essex


Column name changed for 'Falls within' to 'Police Force Region' for better readability

In [7]:
# Better naming conventions for Falls within column
df_street.rename(columns={'Falls within': 'Police Force'}, inplace=True)
print(f"Dataframe-street:\n shape:{df_street.shape} \n column headers new:{df_street.columns.to_list()}" )


Dataframe-street:
 shape:(1468218, 13) 
 column headers new:['Crime ID', 'Month', 'Reported by', 'Police Force', 'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type', 'Last outcome category', 'Context', 'Region']


### Counting and Locating Nans
- A function created, so we can have a look at the initial distribution of all the NaN values, and then call it again after we finished cleaning
- Some columns NaN values are 1-2% of all data across different regions - gives confidence in dropping those rows
- Last Outcome Category column has as much as 22% being NaN values - consider replacing values instead

We can break down dealing with NaNs, into two tasks,
- Task 1: Dropping rows and columns
- Task 2: Replacing NaNs

In [8]:
def nan_pct_by_region(df, region_col='Police Force'):
    """
    Compute the percentage of NaN values in each column, grouped by region.

    Parameters:
      df          : pandas DataFrame
      region_col  : name of the column in df that holds the region value

    Returns:
      A DataFrame where index is each region and the values are the % of missing (NaN) in each column.
    """
    # Compute proportions of NaN per region
    nan_prop = (
        df
        .groupby(region_col)
        .apply(lambda grp: grp.isna().sum() / len(grp))
    )

    # Convert to percentages
    nan_pct = nan_prop * 100

    return nan_pct


In [9]:
# Calling the function to see initial NaN distribution
print(nan_pct_by_region(df_street, region_col='Police Force'))

                             Crime ID  Month  Reported by  Police Force  \
Police Force                                                              
Essex Police                 9.081534    0.0          0.0           0.0   
Hertfordshire Constabulary  21.525720    0.0          0.0           0.0   
Kent Police                 15.261742    0.0          0.0           0.0   
Surrey Police               15.781512    0.0          0.0           0.0   
Thames Valley Police        10.148868    0.0          0.0           0.0   

                            Longitude  Latitude  Location  LSOA code  \
Police Force                                                           
Essex Police                 1.631867  1.631867       0.0   1.631867   
Hertfordshire Constabulary   2.157660  2.157660       0.0   2.157660   
Kent Police                  1.147485  1.147485       0.0   1.147485   
Surrey Police                1.378487  1.378487       0.0   1.378487   
Thames Valley Police         2.139855  2.1

C:\Users\zhary\AppData\Local\Temp\ipykernel_9504\2799959832.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: grp.isna().sum() / len(grp))


### Task 1: Dropping Rows and Columns
 - Drop all the rows with NaN on longatude and latitude
 - Context column has no data. So column is dropped
 - Is there any difference between Police Force Region and Reported By?
    - Check for difference
    - If no difference found, drop the column

In [10]:
# Drop NaN rows for Longitude and Latitude
df_street.dropna(subset=['Longitude','Latitude'], inplace=True)

# Drop Context column
df_street.drop('Context', axis=1, inplace=True)

In [11]:
# Checking difference between 'Police Force Region' and 'Reported by'
count = len(df_street[df_street['Reported by'] != df_street['Police Force']])
print(f"Cases where 'Police Force Region' != 'Reported by':{count}")

Cases where 'Police Force Region' != 'Reported by':0


- No difference found - so column can be dropped without losing data

In [12]:
df_street.drop('Reported by', axis=1, inplace=True)

#### Update on NaNs and their locations
- Now all reside within Crime ID and Last Outcome Category

In [13]:
print(nan_pct_by_region(df_street, region_col='Police Force'))
df_street.head()

                             Crime ID  Month  Police Force  Longitude  \
Police Force                                                            
Essex Police                 9.232191    0.0           0.0        0.0   
Hertfordshire Constabulary  21.998007    0.0           0.0        0.0   
Kent Police                 15.438072    0.0           0.0        0.0   
Surrey Police               16.002099    0.0           0.0        0.0   
Thames Valley Police        10.224356    0.0           0.0        0.0   

                            Latitude  Location  LSOA code  LSOA name  \
Police Force                                                           
Essex Police                     0.0       0.0        0.0        0.0   
Hertfordshire Constabulary       0.0       0.0        0.0        0.0   
Kent Police                      0.0       0.0        0.0        0.0   
Surrey Police                    0.0       0.0        0.0        0.0   
Thames Valley Police             0.0       0.0        0.

C:\Users\zhary\AppData\Local\Temp\ipykernel_9504\2799959832.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: grp.isna().sum() / len(grp))


,Crime ID,Month,Police Force,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Region
0,NaN,2023-10,Essex Police,0.439004,51.641536,On or near Derby Close,E01021237,Basildon 001A,Anti-social behaviour,NaN,essex
1,663fc84dec3c68ead7dbdc274183432ad72008d9c1eb89...,2023-10,Essex Police,0.439614,51.639167,On or near Smythe Road,E01021237,Basildon 001A,Other theft,Investigation complete; no suspect identified,essex
2,446d4dc29575f600213a210fe477eb2dee5ccad548b3f2...,2023-10,Essex Police,0.432091,51.640106,On or near Oakwood Drive,E01021237,Basildon 001A,Violence and sexual offences,Further action is not in the public interest,essex
3,04246cd15ed95c5a8db98467158ff84c16a3944042300e...,2023-10,Essex Police,0.435584,51.640825,On or near Doublet Mews,E01021238,Basildon 001B,Vehicle crime,Investigation complete; no suspect identified,essex
4,1d720ddc539b23c226a96b67a52a9cba67b89007cd6230...,2023-10,Essex Police,0.423868,51.644206,On or near Sussex Way,E01021250,Basildon 001D,Violence and sexual offences,Unable to prosecute suspect,essex


### Task 2: Replacing NaNs
- Last Outcome Category Column
    - check the values found within the column
    - replace NaNs with an appropriate category

In [14]:
# Check the Category column
df_street['Last outcome category'].value_counts(dropna=False)

Last outcome category
Unable to prosecute suspect                            488331
Investigation complete; no suspect identified          454058
NaN                                                    197801
Under investigation                                     64544
Court result unavailable                                59031
Status update unavailable                               51012
Local resolution                                        40547
Awaiting court outcome                                  32667
Action to be taken by another organisation              28147
Offender given a caution                                11853
Further investigation is not in the public interest      5579
Formal action is not in the public interest              4847
Further action is not in the public interest             3997
Suspect charged as part of another case                   691
Offender given penalty notice                             239
Offender given a drugs possession warning       

- Given the different categories, NaN values to be categorized as 'Status update unavailable'

In [15]:
df_street['Last outcome category'] = df_street['Last outcome category'].fillna('Status update unavailable')
df_street['Last outcome category'].value_counts(dropna=False)


Last outcome category
Unable to prosecute suspect                            488331
Investigation complete; no suspect identified          454058
Status update unavailable                              248813
Under investigation                                     64544
Court result unavailable                                59031
Local resolution                                        40547
Awaiting court outcome                                  32667
Action to be taken by another organisation              28147
Offender given a caution                                11853
Further investigation is not in the public interest      5579
Formal action is not in the public interest              4847
Further action is not in the public interest             3997
Suspect charged as part of another case                   691
Offender given penalty notice                             239
Offender given a drugs possession warning                  11
Name: count, dtype: int64

### Check For anomales
- Here we are checking for anomales whereby, the geospatial data of an entry does not lie within the Police Force region it was reported in. This is done by doing the following
    - Loading the boundary data of each Police force region
    - Checking every point, if it lies within the appropriate region. Otherwise, it is filtered out

In [16]:
# Checking if its within regions of interest
geometry = [Point(xy) for xy in zip(df_street['Longitude'], df_street['Latitude'])]
gdf_points = gpd.GeoDataFrame(df_street, geometry=geometry, crs="EPSG:4326")

# Spatial join to keep only points within any region polygon
gdf_filtered = gpd.sjoin(gdf_points, final_gdf, predicate='within', how='inner')

# Drop geometry columns after use
df_street = gdf_filtered.drop(columns=['geometry'])

## Final Check of cleaned data
- Remaining NaNs lie within Crime ID column
    - Chosen to keep rows with that data as takes upto 1/4 of rows for some Policing regions
    - Analysis can be continued with Crime IDs missing

In [17]:
print(f"Before Cleaning \n shape:{df_street_raw.shape}")
print(df_street_raw.isna().sum())
print(f"\nAfter Cleaning: \n shape:{df_street.shape}")
print(df_street.isna().sum())

df_street.head()


Before Cleaning 
 shape:(1468218, 13)
Crime ID                  198375
Month                          0
Reported by                    0
Falls within                   0
Longitude                  24863
Latitude                   24863
Location                       0
LSOA code                  24863
LSOA name                  24863
Crime type                     0
Last outcome category     198375
Context                  1468218
Region                         0
dtype: int64

After Cleaning: 
 shape:(1440238, 14)
Crime ID                 197451
Month                         0
Police Force                  0
Longitude                     0
Latitude                      0
Location                      0
LSOA code                     0
LSOA name                     0
Crime type                    0
Last outcome category         0
Region                        0
index_right                   0
Name                          0
Description                   0
dtype: int64


,Crime ID,Month,Police Force,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Region,index_right,Name,Description
0,NaN,2023-10,Essex Police,0.439004,51.641536,On or near Derby Close,E01021237,Basildon 001A,Anti-social behaviour,Status update unavailable,essex,0,Essex Police,
1,663fc84dec3c68ead7dbdc274183432ad72008d9c1eb89...,2023-10,Essex Police,0.439614,51.639167,On or near Smythe Road,E01021237,Basildon 001A,Other theft,Investigation complete; no suspect identified,essex,0,Essex Police,
2,446d4dc29575f600213a210fe477eb2dee5ccad548b3f2...,2023-10,Essex Police,0.432091,51.640106,On or near Oakwood Drive,E01021237,Basildon 001A,Violence and sexual offences,Further action is not in the public interest,essex,0,Essex Police,
3,04246cd15ed95c5a8db98467158ff84c16a3944042300e...,2023-10,Essex Police,0.435584,51.640825,On or near Doublet Mews,E01021238,Basildon 001B,Vehicle crime,Investigation complete; no suspect identified,essex,0,Essex Police,
4,1d720ddc539b23c226a96b67a52a9cba67b89007cd6230...,2023-10,Essex Police,0.423868,51.644206,On or near Sussex Way,E01021250,Basildon 001D,Violence and sexual offences,Unable to prosecute suspect,essex,0,Essex Police,


- Saving cleaned street data

In [20]:
# Save cleaned dataframe to new csv file
output_cleaned_path = os.path.join("cleaned-data", "cleaned_street.csv")
df_street.to_csv(output_cleaned_path, index=False)
print(f"Cleaned street data saved to: {output_cleaned_path}")

Cleaned street data saved to: cleaned-data\cleaned_street.csv


## Dataframe - Stop and Search
---------------
- Copy of raw data created

In [21]:
df_stop_and_search_raw = df_stop_and_search.copy()
print(f"Dataframe-stop_and_search:\n shape:{df_stop_and_search.shape} \n column headers:{df_stop_and_search.columns.to_list()}" )

## Preview of Dataframe - Street
df_stop_and_search.head()

Dataframe-stop_and_search:
 shape:(118298, 16) 
 column headers:['Type', 'Date', 'Part of a policing operation', 'Policing operation', 'Latitude', 'Longitude', 'Gender', 'Age range', 'Self-defined ethnicity', 'Officer-defined ethnicity', 'Legislation', 'Object of search', 'Outcome', 'Outcome linked to object of search', 'Removal of more than just outer clothing', 'Region']


,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,Region
0,Person search,2023-09-30T23:10:00+00:00,False,NaN,51.580660,0.530809,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),NaN,A no further action disposal,NaN,NaN,essex
1,Person search,2023-09-30T23:10:00+00:00,False,NaN,51.712312,0.542063,Female,25-34,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),NaN,Community resolution,NaN,NaN,essex
2,Vehicle search,2023-09-30T23:10:00+00:00,False,NaN,51.544994,0.716064,NaN,NaN,NaN,NaN,Police and Criminal Evidence Act 1984 (section 1),NaN,A no further action disposal,NaN,NaN,essex
3,Vehicle search,2023-09-30T23:10:00+00:00,False,NaN,51.544994,0.716064,NaN,NaN,NaN,NaN,Police and Criminal Evidence Act 1984 (section 1),NaN,A no further action disposal,NaN,NaN,essex
4,Person search,2023-09-30T23:10:00+00:00,False,NaN,51.544994,0.716064,Female,25-34,Other ethnic group - Not stated,White,Police and Criminal Evidence Act 1984 (section 1),NaN,A no further action disposal,NaN,NaN,essex


### Counting and Locating Nans

In [22]:
# Calling the function to see initial NaN distribution
print(nan_pct_by_region(df_stop_and_search, region_col='Region'))


               Type  Date  Part of a policing operation  Policing operation  \
Region                                                                        
essex           0.0   0.0                           0.0               100.0   
hertfordshire   0.0   0.0                           0.0               100.0   
kent            0.0   0.0                           0.0               100.0   
surrey          0.0   0.0                           0.0               100.0   
thames-valley   0.0   0.0                           0.0               100.0   

                Latitude  Longitude    Gender  Age range  \
Region                                                     
essex           0.787826   0.787826  8.157700  23.533769   
hertfordshire  18.105956  18.105956  4.072883   8.107487   
kent            0.875331   0.875331  4.396250  10.680341   
surrey          0.527333   0.527333  2.214800  10.414836   
thames-valley   6.613197   6.613197  2.206387   7.796893   

               Self-defin

C:\Users\zhary\AppData\Local\Temp\ipykernel_9504\2799959832.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: grp.isna().sum() / len(grp))


## Longitude and Latitude data
- Hertfordshire has almost 1/5th of its entries missing geospatial data
- We do not want to get rid of this data, therefore the following steps will be made for Longitude and Latitude columns
    1. Identify missing values 
    2. Find "similar" rows
    3. Apply noise to avoid over plotting
    4. Mark rows for which data was made up

In [23]:
# Identify missing values
mask_missing = df_stop_and_search['Latitude'].isna() | df_stop_and_search['Longitude'].isna()
df_stop_and_search['was_imputed'] = False
# Define match levels
levels = [
    ['Region', 'Type', 'Policing operation', 'Date'],
    ['Region', 'Type','Date'],
    ['Region', 'Type'],
    ['Region']
]

# Here we are adding another column to keep track of what variables were used for imputation
donor_maps = {}
df_stop_and_search['group_key'] = ''  

for level_idx, lvl in enumerate(levels, start=1):
    key = '_'.join(lvl)
    # Set the group_key column for this level
    df_stop_and_search['group_key'] = df_stop_and_search[lvl].astype(str).agg('_'.join, axis=1)

    # Build donor map using this single group_key
    donors = (
        df_stop_and_search.loc[~mask_missing]
        .groupby('group_key')[['Latitude', 'Longitude']]
        .median()
        .rename(columns={
            'Latitude': f'lat_donor_{key}',
            'Longitude': f'lon_donor_{key}'
        })
    )

    donor_maps[key] = donors

# For any values still missing, we fill with the median
if mask_missing.any():
    # Compute region‐wise medians
    region_medians = (
        df_stop_and_search.loc[~mask_missing]
          .groupby('Region')[['Latitude','Longitude']]
          .median()
          .rename(columns={'Latitude':'lat_med_region',
                           'Longitude':'lon_med_region'})
    )
    df_stop_and_search = df_stop_and_search.join(region_medians, on='Region')

# Now fill missing — but only if region median exists.
    fill_mask_region = mask_missing & df_stop_and_search['lat_med_region'].notna()

    df_stop_and_search.loc[fill_mask_region, ['Latitude','Longitude']] = (
        df_stop_and_search.loc[fill_mask_region, ['lat_med_region','lon_med_region']].values
    )
    df_stop_and_search.loc[fill_mask_region, 'was_imputed']      = True

    # For any still missing (region median missing) fall back to overall median
    mask_still_missing = df_stop_and_search['Latitude'].isna() | df_stop_and_search['Longitude'].isna()
    if mask_still_missing.any():
        med_lat = df_stop_and_search['Latitude'].median()
        med_lon = df_stop_and_search['Longitude'].median()
        df_stop_and_search.loc[mask_still_missing, ['Latitude','Longitude']] = [med_lat, med_lon]
        df_stop_and_search.loc[mask_still_missing, 'was_imputed'] = True
        df_stop_and_search.loc[mask_still_missing, 'imputation_level'] = 'global_median'

    # Clean up region_medians join columns
    df_stop_and_search = df_stop_and_search.drop(columns=['lat_med_region','lon_med_region'])

#Apply noise to avoid overplotting
noise_scale = 0.00000001
imputed_mask = df_stop_and_search['was_imputed'] == True
df_stop_and_search.loc[imputed_mask, 'Latitude']  += np.random.normal(scale=noise_scale, size=imputed_mask.sum())
df_stop_and_search.loc[imputed_mask, 'Longitude'] += np.random.normal(scale=noise_scale, size=imputed_mask.sum())
cleanup_cols = [col for col in df_stop_and_search.columns if col.startswith('group_key_') or col.startswith('lat_donor_') or col.startswith('lon_donor_')]
df = df_stop_and_search.drop(columns=cleanup_cols, errors=True)


In [24]:
df_stop_and_search.columns

Index(['Type', 'Date', 'Part of a policing operation', 'Policing operation',
       'Latitude', 'Longitude', 'Gender', 'Age range',
       'Self-defined ethnicity', 'Officer-defined ethnicity', 'Legislation',
       'Object of search', 'Outcome', 'Outcome linked to object of search',
       'Removal of more than just outer clothing', 'Region', 'was_imputed',
       'group_key'],
      dtype='object')

- The following two cells are a check on the shape and count of made up geospatial data

In [25]:
df_stop_and_search.shape

(118298, 18)

In [26]:
df_stop_and_search.value_counts('was_imputed')

was_imputed
False    113153
True       5145
Name: count, dtype: int64

### Updated look on NaN locations

In [27]:
print(nan_pct_by_region(df_stop_and_search, region_col='Region'))

               Type  Date  Part of a policing operation  Policing operation  \
Region                                                                        
essex           0.0   0.0                           0.0               100.0   
hertfordshire   0.0   0.0                           0.0               100.0   
kent            0.0   0.0                           0.0               100.0   
surrey          0.0   0.0                           0.0               100.0   
thames-valley   0.0   0.0                           0.0               100.0   

               Latitude  Longitude    Gender  Age range  \
Region                                                    
essex               0.0        0.0  8.157700  23.533769   
hertfordshire       0.0        0.0  4.072883   8.107487   
kent                0.0        0.0  4.396250  10.680341   
surrey              0.0        0.0  2.214800  10.414836   
thames-valley       0.0        0.0  2.206387   7.796893   

               Self-defined ethn

C:\Users\zhary\AppData\Local\Temp\ipykernel_9504\2799959832.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: grp.isna().sum() / len(grp))


- After using other columns to find donor data we can now can drop all the other columns that we do not need for the analysis

In [28]:
df_stop_and_search.drop(columns=['Part of a policing operation',
                                 'Policing operation',
                                 'Gender',
                                 'Age range',
                                 'Self-defined ethnicity',
                                 'Officer-defined ethnicity',
                                 'Legislation',
                                 'Object of search',
                                 'Outcome',
                                 'Outcome linked to object of search',
                                 'Removal of more than just outer clothing'
                                 ], inplace=True)

- Similar to our street data, we also need to filter out points which occur outside the region they are supposed to be covered by

In [29]:
# Converting df_stop_and_search into GeoDataFrame
geometry = [Point(xy) for xy in zip(df_stop_and_search['Longitude'], df_stop_and_search['Latitude'])]
gdf_points = gpd.GeoDataFrame(df_stop_and_search, geometry=geometry, crs="EPSG:4326")

# Spatial join to keep only points within any region polygon
gdf_filtered = gpd.sjoin(gdf_points, final_gdf, predicate='within', how='inner')

# Drop geometry columns or keep as needed
df_stop_and_search = gdf_filtered.drop(columns=['geometry'])

- Renaming 'Name' to 'Police Force' to match street data

In [30]:
df_stop_and_search["Police Force"] = df_stop_and_search["Name"].str.strip()
df_stop_and_search.drop(columns=["Name"], inplace=True)

- Saving cleaned Stop and Search data to a new csv

In [31]:
output_cleaned_path = os.path.join("cleaned-data", "cleaned_stop_and_search.csv")
df_stop_and_search.to_csv(output_cleaned_path, index=False)
print(f"Cleaned stop-and-search data saved to: {output_cleaned_path}")

Cleaned stop-and-search data saved to: cleaned-data\cleaned_stop_and_search.csv
